# Tugas Clustering 

##### Ilham Muhammad Misbahuddin
##### 05111540000088

## Kebutuhan :
1. Operating System : Kali Linux 2019.1
2. Apache Spark 2.3.3
3. Scala 2.12.8
4. Python 3.7.3rc1
5. PySpark 2.4.0
6. Findspark 1.3.0
7. Jupyter 4.4.0

## Deskripsi Dataset
* Nama Dataset : [Online Retail](https://www.kaggle.com/puneetbhaya/online-retail)

<table>
    <thead>
        <tr>
            <th>Sumber Data</th>
            <th>Jumlah Baris</th>
            <th>Jumlah Colom</th>
            <th>Ukuran</th>
            <th>Format File</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>Online Retail.xlsx</td>
            <td>541909</td>
            <td>8</td>
            <td>23,7 MB</td>
            <td>XLSX</td>
        </tr>
    </tbody>
</table>
    


## Inisialisasi Apache Spark

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

# Create Spark Session
spark = SparkSession.builder.appName("Big Data Frequent Itemsets").getOrCreate()

print(spark)

## Load Dataset

In [2]:
# Load excel
import pandas as pd
from pyspark.sql.types import *

df_excel = pd.read_excel("/root/Lecture/BIGDATA/datasets/Online Retail.xlsx")

header = StructType([ StructField("InvoiceNo", StringType(), True)\
                       ,StructField("StockCode", StringType(), True)\
                       ,StructField("Description", StringType(), True)\
                       ,StructField("Quantity", IntegerType(), True)\
                       ,StructField("InvoiceDate", StringType(), True)\
                       ,StructField("UnitPrice", DoubleType(), True)\
                       ,StructField("CustomerID", StringType(), True)\
                       ,StructField("Country", StringType(), True)])

df = spark.createDataFrame(df_excel, schema=header)
type(df)

pyspark.sql.dataframe.DataFrame

In [3]:
# Count dataset Online Retail
df.count()

541909

In [4]:
# Show dataset Online Retail
df.show()

+---------+---------+--------------------+--------+--------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|         InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|java.util.Gregori...|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|java.util.Gregori...|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|java.util.Gregori...|     2.75|   17850.0|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|java.util.Gregori...|     3.39|   17850.0|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|java.util.Gregori...|     3.39|   17850.0|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|java.util.Gregori...|     7.65|   17850.0|United Kingdom|
|   536365|    2173

In [5]:
# Drop some column
dropped_column = ['Quantity', 'InvoiceDate', 'UnitPrice', 'Country', 'Description', 'CustomerID']
df = df.drop(*dropped_column)

# Show dataset after some column dropped
df.show()

+---------+---------+
|InvoiceNo|StockCode|
+---------+---------+
|   536365|   85123A|
|   536365|    71053|
|   536365|   84406B|
|   536365|   84029G|
|   536365|   84029E|
|   536365|    22752|
|   536365|    21730|
|   536366|    22633|
|   536366|    22632|
|   536367|    84879|
|   536367|    22745|
|   536367|    22748|
|   536367|    22749|
|   536367|    22310|
|   536367|    84969|
|   536367|    22623|
|   536367|    22622|
|   536367|    21754|
|   536367|    21755|
|   536367|    21777|
+---------+---------+
only showing top 20 rows



In [6]:
# Group dataset 
from pyspark.sql import functions as F

gdf = df.groupby("InvoiceNo").agg(F.collect_set("StockCode"))

# Show groupped dataset
gdf.show()

+---------+----------------------+
|InvoiceNo|collect_set(StockCode)|
+---------+----------------------+
|   536596|  [22900, 22114, 84...|
|   536938|  [22112, 21931, 84...|
|   537252|               [22197]|
|   537691|  [22505, 46000R, 2...|
|   538041|               [22145]|
|   538184|  [22561, 22147, 21...|
|   538517|  [22749, 21212, 22...|
|   538879|  [21212, 22759, 22...|
|   539275|  [22083, 22150, 22...|
|   539630|  [22111, 22971, 22...|
|   540499|  [22697, 22796, 21...|
|   540540|  [22111, 22834, 22...|
|   540976|  [22413, 21212, 22...|
|   541432|  [22113, 22457, 21...|
|   541518|  [21212, 22432, 22...|
|   541783|  [22561, 22697, 22...|
|   542026|  [22398, 22194, 22...|
|   542375|  [22629, 21731, 22...|
|   543641|  [22645, 75131, 22...|
|   544303|  [84596L, 22931, 8...|
+---------+----------------------+
only showing top 20 rows



## Prediksi Frequent Itemsets

In [ ]:
# Frequent itemsets with Minimum Support = 0.01 and Minimum Confidence = 0.8
from pyspark.ml.fpm import FPGrowth

fpGrowth = FPGrowth(itemsCol="collect_set(StockCode)", minSupport=0.01, minConfidence=0.8)
model = fpGrowth.fit(gdf)
model.freqItemsets.show()

In [ ]:
# Association rule with Minimum Support = 0.01 and Minimum Confidence = 0.8
model.associationRules.show()

In [ ]:
# Item prediction with Minimum Support = 0.01 and Minimum Confidence = 0.8
model.transform(gdf).show()

In [ ]:
# Frequent itemsets with Minimum Support = 0.02 and Minimum Confidence = 0.7
fpGrowth = FPGrowth(itemsCol="collect_set(StockCode)", minSupport=0.02, minConfidence=0.7)
model = fpGrowth.fit(gdf)
model.freqItemsets.show()

In [ ]:
# Association rule with Minimum Support = 0.02 and Minimum Confidence = 0.7
model.associationRules.show()

In [ ]:
# Item prediction with Minimum Support = 0.02 and Minimum Confidence = 0.7
model.transform(gdf).show()

In [ ]:
# Frequent itemsets with Minimum Support = 0.03 and Minimum Confidence = 0.8
fpGrowth = FPGrowth(itemsCol="collect_set(StockCode)", minSupport=0.03, minConfidence=0.6)
model = fpGrowth.fit(gdf)
model.freqItemsets.show()

In [ ]:
# Association rule with Minimum Support = 0.03 and Minimum Confidence = 0.6
model.associationRules.show()

In [ ]:
# Item prediction with Minimum Support = 0.03 and Minimum Confidence = 0.6
model.transform(gdf).show()